### desc2idの作成  
・24/7/29

In [1]:
# num = str(2)
# num = str(3)
# num = str(4)
num = str(5)

In [ ]:
from src.mymodule import read_bin, save_bin
import pandas as pd

path = "DATA/jpbook_and_BR/cross-valid"
train = pd.read_parquet(path + '/train_'+num+'.parquet')#10293
test = pd.read_parquet(path + '/test_'+num+'.parquet')#10293
train_index = train.index#7352
test_index = test.index#1838

In [5]:
# 名称と対応するtrainの行名セットを対にした辞書
desc2id_d = dict()
for i in train_index:
    name = train["description"][i]
    if name not in desc2id_d:
        desc2id_d[name] = set()
    desc2id_d[name].add(i)
# akaashishigi:{0, 1, 2, 3, 4, 5}

In [147]:
# save_bin("new_data/desc2id.bin",desc2id_d)

In [6]:
names = list(set(list(train["description"])))

In [149]:
del train,test

### id2emb

In [7]:
def normalize(vector):
    norm = np.linalg.norm(vector, axis=-1, keepdims=True)
    return vector / (norm + 1e-9)  # avoid division by zero

In [9]:
import torch#1.12.1+cu113
import torchaudio
from transformers import Wav2Vec2ForPreTraining, Wav2Vec2FeatureExtractor
import os
from contextlib import redirect_stdout
import numpy as np


with redirect_stdout(open(os.devnull, 'w')):#一時的に出力を無効化
    
    # モデルの読み込み
    path = "./models/cv"+str(num)+"/"
    model = Wav2Vec2ForPreTraining.from_pretrained(path)# Wav2Vec2ForCTC => Wav2Vec2ForPreTraining
    # kojima-r/BirdWav2Vec-train クローンの別プロジェクトでモデルを作成・移動

    # トークナイザのパスを指定してProcessorをロード
    processor = Wav2Vec2FeatureExtractor.from_pretrained("patrickvonplaten/wav2vec2-base-v2")# Wav2Vec2Processor => Wav2Vec2FeatureExtractor
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

In [10]:
import torch
import io
import scipy.io.wavfile as wavfile
from transformers import Wav2Vec2ForCTC
import numpy as np

id2emb_processor = dict()
id2emb = dict()
id2emb_q = dict()

c = 0
for i in train_index:
# for i in range(3):
    print('\r%d / %d' %(c, len(train_index)), end='')
    c += 1
    
    # WAVファイルの読み込み
    sample_rate, waveform = wavfile.read('wav/'+str(i)+'.wav')

    # 前処理
    input_values = processor(waveform, sampling_rate=sample_rate, return_tensors="pt").input_values
    input_values = input_values.to(torch.float32)
    
    input_values = input_values.to(device)# GPUに送る
    id2emb_processor[i] = input_values.cpu().numpy()#保存

    # モデルに入力し、出力を取得
    with torch.no_grad():
        result=model(input_values)
        
        embedding = result.projected_states.detach().cpu().numpy()
        embedding_q = result.projected_quantized_states.detach().cpu().numpy()
        
        # 通常
        id2emb[i] = embedding
        id2emb_q[i] = embedding_q
        
        # 正規化
        # id2emb[i] = normalize(embedding)
        # id2emb_q[i] = normalize(embedding_q)

7351 / 7352

In [12]:
batch_size = 8

In [13]:
save_bin("new_data_cv"+num+"/id2emb_processor_b"+str(batch_size)+".bin",id2emb_processor)
save_bin("new_data_cv"+num+"/id2emb_b"+str(batch_size)+".bin",id2emb)
save_bin("new_data_cv"+num+"/id2emb_q_b"+str(batch_size)+".bin",id2emb_q)

In [14]:
import torch
import io
import scipy.io.wavfile as wavfile
from transformers import Wav2Vec2ForCTC
import numpy as np

id2emb_processor = dict()
id2emb = dict()
id2emb_q = dict()

c = 0
for i in test_index:
# for i in range(3):
    print('\r%d / %d' %(c, len(test_index)), end='')
    c += 1
    
    # WAVファイルの読み込み
    sample_rate, waveform = wavfile.read('wav/'+str(i)+'.wav')

    # 前処理
    input_values = processor(waveform, sampling_rate=sample_rate, return_tensors="pt").input_values
    input_values = input_values.to(torch.float32)
    
    input_values = input_values.to(device)# GPUに送る
    id2emb_processor[i] = input_values.cpu().numpy()#保存

    # モデルに入力し、出力を取得
    with torch.no_grad():
        result=model(input_values)
        
        embedding = result.projected_states.detach().cpu().numpy()
        embedding_q = result.projected_quantized_states.detach().cpu().numpy()
        
        # 通常
        id2emb[i] = embedding
        id2emb_q[i] = embedding_q
        
        # 正規化
        # id2emb[i] = normalize(embedding)
        # id2emb_q[i] = normalize(embedding_q)

1837 / 1838

In [15]:
save_bin("new_data_cv"+num+"/test_id2emb_processor_b"+str(batch_size)+".bin",id2emb_processor)
save_bin("new_data_cv"+num+"/test_id2emb_b"+str(batch_size)+".bin",id2emb)
save_bin("new_data_cv"+num+"/test_id2emb_q_b"+str(batch_size)+".bin",id2emb_q)

### WavDesc2jid

In [16]:
name2jid = read_bin("new_data/name2jid.bin")#01にて作成
WavDesc2Kana = read_bin("new_data/WavDesc2Kana.bin")#01にて作成

In [17]:
# 単純なカナ一致で何とかなる音声名
WavDesc2jid = {}
fail = []
for Desc,kana in WavDesc2Kana.items():
    if kana in name2jid:
        WavDesc2jid[Desc] = name2jid[kana]
    else:
        fail.append(Desc)

In [ ]:
# 単純なカナ一致で何とかなる音声名
WavDesc2jid = {}
fail = []
for Desc,kana in WavDesc2Kana.items():
    if kana in name2jid:
        WavDesc2jid[Desc] = name2jid[kana]
    else:
        fail.append(Desc)

import csv
fix_d = dict()
# 手動で紐づけた音声名
with open("error_fix_2.csv", newline='',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row[2] != "":
            fix_d[row[0]] = row[2]

# WavDesc2jid = {**WavDesc2jid, **fix_d}
save_bin("new_data_cv"+num+"/WavDesc2jid.bin",WavDesc2jid)

In [19]:
# WavDesc2jid = {**WavDesc2jid, **fix_d}
save_bin("new_data_cv"+num+"/WavDesc2jid.bin",WavDesc2jid)

In [20]:
del name2jid,WavDesc2Kana,WavDesc2jid

### bidとSvecをつなげる

In [21]:
name2jid = read_bin("new_data/name2jid.bin")#01にて作成
WavDesc2jid = read_bin("new_data_cv"+num+"/WavDesc2jid.bin")

In [22]:
jid2sound_idx = dict()#366/550
for desc,s_idx_set in desc2id_d.items():
    if desc in WavDesc2jid:
        jid = WavDesc2jid[desc]
        jid2sound_idx[jid] = s_idx_set
        
save_bin("new_data_cv"+num+"/jid2sound_idx.bin",jid2sound_idx)
del name2jid,WavDesc2jid

In [23]:
bid_jid = read_bin('new_data/tuples_bid_jid.bin')

In [24]:
bid2sound_idx = dict()

for bid,jid in bid_jid:
    if jid in jid2sound_idx:
        sound_idx = jid2sound_idx[jid]
        if bid not in bid2sound_idx:
            bid2sound_idx[bid]=set()
        bid2sound_idx[bid].update(sound_idx)
# save_bin("new_data/bid2sound_idx.bin",bid2sound_idx)

In [25]:
id2emb_p = read_bin("new_data_cv"+num+"/id2emb_processor_b8.bin")
id2emb = read_bin("new_data_cv"+num+"/id2emb_b8.bin")

In [26]:
def Vecs_tolist(vec,processor_mode):
    out=[]
    v=np.mean(vec,axis=1)
    return v.tolist()[0]

In [27]:
bid2emb = dict()
bid2emb_p = dict()


c = 1
for bid,sound_idx_set in bid2sound_idx.items():
    bid2emb[bid] = []
    bid2emb_p[bid] = []
    for idx in sound_idx_set:
        bid2emb[bid].append(Vecs_tolist(id2emb[idx],False))
        bid2emb_p[bid].append(id2emb_p[idx].tolist()[0])

In [28]:
save_bin("new_data_cv"+num+"/bid2emb.bin",bid2emb)
save_bin("new_data_cv"+num+"/bid2emb_p.bin",bid2emb_p)

In [29]:
del bid2sound_idx,bid_jid,jid2sound_idx

### concat_vec

In [30]:
bid2Gvec = read_bin('new_data/bid2Gvec.bin')

bid_eid = read_bin('new_data/tuples_bid_eid.bin')
bid_jid = read_bin('new_data/tuples_bid_jid.bin')

eid2Lvec = read_bin('new_data/eid2Lvec.bin')
jid2Lvec = read_bin('new_data/jid2Lvec.bin')

In [31]:
bid2emb = read_bin("new_data_cv"+num+"/bid2emb.bin")
bid2emb_p = read_bin("new_data_cv"+num+"/bid2emb_p.bin")

In [32]:
en_concat_vecs = []
en_concat_vecs_p = []

i = 0
for bid,eid in bid_eid:
    print('\r%d / %d' %(i, len(bid_eid)), end='')
    i = i+1
    
    Lvec = [0]*768
    Gvec = [0]*64
    Svec = [0]*256
    Svec_p = []
    
    NoData = True
    if eid in eid2Lvec:
        Lvec = eid2Lvec[eid]
        
    if bid in bid2Gvec:
        Gvec = bid2Gvec[bid]
        
    if bid in bid2emb:
        for emb in bid2emb[bid]:
            Svec = emb
            concat_vec = Lvec + Gvec + Svec
            en_concat_vecs.append([bid,concat_vec])
    else:
        concat_vec = Lvec + Gvec + Svec
        en_concat_vecs.append([bid,concat_vec])
            
    if bid in bid2emb_p:
        for emb in bid2emb_p[bid]:
            Svec_p = emb
            concat_vec_p = Lvec + Gvec + Svec_p
            en_concat_vecs_p.append([bid,concat_vec_p])
    else:
        concat_vec_p = Lvec + Gvec + Svec_p
        en_concat_vecs_p.append([bid,concat_vec_p])

70236 / 70237

In [33]:
ja_concat_vecs = []
ja_concat_vecs_p = []

i = 0
for bid,jid in bid_jid:
    print('\r%d / %d' %(i, len(bid_jid)), end='')
    i = i+1
    
    Lvec = [0]*768
    Gvec = [0]*64
    Svec = [0]*256
    Svec_p = []
    
    NoData = True
    if jid in jid2Lvec:
        Lvec = jid2Lvec[jid]
        
    if bid in bid2Gvec:
        Gvec = bid2Gvec[bid]
        
    if bid in bid2emb:
        for emb in bid2emb[bid]:
            Svec = emb
            concat_vec = Lvec + Gvec + Svec
            ja_concat_vecs.append([bid,concat_vec])
    else:
        concat_vec = Lvec + Gvec + Svec
        ja_concat_vecs.append([bid,concat_vec])
            
    if bid in bid2emb_p:
        for emb in bid2emb_p[bid]:
            Svec_p = emb
            concat_vec_p = Lvec + Gvec + Svec_p
            ja_concat_vecs_p.append([bid,concat_vec_p])
    else:
        concat_vec_p = Lvec + Gvec + Svec_p
        ja_concat_vecs_p.append([bid,concat_vec_p])

10714 / 10715

In [34]:
save_bin('new_data_cv'+num+'/en_concat_vecs.bin',en_concat_vecs)
save_bin('new_data_cv'+num+'/en_concat_vecs_p.bin',en_concat_vecs_p)
save_bin('new_data_cv'+num+'/ja_concat_vecs.bin',ja_concat_vecs)
save_bin('new_data_cv'+num+'/ja_concat_vecs_p.bin',ja_concat_vecs_p)

In [35]:
print("momo")

momo
